In [3]:
import pandas as pd

In [2]:
!dir

 Volume in drive D is Data
 Volume Serial Number is 86AA-243D

 Directory of d:\Binar Academy - Data Science\challenge_platinum

16/02/2023  19:25    <DIR>          .
16/02/2023  19:25    <DIR>          ..
16/02/2023  19:18         2.186.718 train_preprocess.tsv.txt
               1 File(s)      2.186.718 bytes
               2 Dir(s)  169.941.155.840 bytes free


In [5]:
df = pd.read_csv("train_preprocess.tsv.txt", sep="\t", engine="python", names=["data", "label"])

In [6]:
df

,data,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [11]:
df.shape

(11000, 2)

In [13]:
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.corpus.stopwords.words('indonesian')

def cleansing(text):
    # Make sentence being lowercase
    text = text.lower()

    # Remove user, rt, \n, retweet, \t, url, xd
    pattern_1 = r'(user|retweet|\\t|\\r|url|xd)'
    text = re.sub(pattern_1, '', text)

    # Remove mention
    pattern_2 = r'@[^\s]+'
    text = re.sub(pattern_2, '', text)

    # Remove hashtag
    pattern_3 = r'#([^\s]+)'
    text = re.sub(pattern_3, '', text)

    # Remove general punctuation, math operation char, etc.
    pattern_4 = r'[\,\@\*\_\-\!\:\;\?\'\.\"\)\(\{\}\<\>\+\%\$\^\#\/\`\~\|\&\|]'
    text = re.sub(pattern_4, ' ', text)

    # Remove single character
    pattern_5 = r'\b\w{1,3}\b'
    text = re.sub(pattern_5, '', text)

    # Remove emoji
    pattern_6 = r'\\[a-z0-9]{1,5}'
    text = re.sub(pattern_6, '', text)

    # Remove digit character
    pattern_7 = r'\d+'
    text = re.sub(pattern_7, '', text)

    # Remove url start with http or https
    pattern_8 = r'(https|https:)'
    text = re.sub(pattern_8, '', text)

    # Remove (\); ([); (])
    pattern_9 = r'[\\\]\[]'
    text = re.sub(pattern_9, '', text)

    # Remove character non ASCII
    pattern_10 = r'[^\x00-\x7f]'
    text = re.sub(pattern_10, '', text)

    # Remove character non ASCII
    pattern_11 = r'(\\u[0-9A-Fa-f]+)'
    text = re.sub(pattern_11, '', text)

    # Remove multiple whitespace
    pattern_12 = r'(\s+|\\n)'
    text = re.sub(pattern_12, ' ', text)
    
    # Remove whitespace at the first and end sentences
    text = text.rstrip()
    text = text.lstrip()
    return text

indo_stop_words = stopwords.words("indonesian")

def remove_stopwords(text):
    return ' '.join([word for word in word_tokenize(text) if word not in indo_stop_words])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
df_coba = df.applymap(cleansing)
df_coba = df_coba.applymap(remove_stopwords)
df_coba

,data,label
0,warung dimiliki pengusaha pabrik puluhan terke...,positive
1,mohon ulama lurus k mmbri hujjah partai diwlh ...,neutral
2,lokasi strategis jalan sumatera bandung nyaman...,positive
3,betapa bahagia unboxing paket barang bagus men...,positive
4,mahasiswa sombong kasih kartu kuning belajar u...,negative
...,...,...
10995,kecewa,positive
10996,enak masakan kepiting menyenangkan memilih kep...,positive
10997,hormati partai partai berkoalisi,neutral
10998,pagi pagi pasteur macet parah bikin jengkel,negative


In [18]:
df_coba[['data']][0:10]

,data
0,warung dimiliki pengusaha pabrik puluhan terke...
1,mohon ulama lurus k mmbri hujjah partai diwlh ...
2,lokasi strategis jalan sumatera bandung nyaman...
3,betapa bahagia unboxing paket barang bagus men...
4,mahasiswa sombong kasih kartu kuning belajar u...
5,makanan beragam harga makanan food stall kasir...
6,pakai kartu kredit untung rugi
7,unik bagus foto makanan enak pegawai ramah ber...
8,keluarga menikmati pengalaman kuliner menyenan...
9,bersyukur
